<a href="https://colab.research.google.com/github/fhswf/NLP_BERT/blob/master/NER_BERT_GermEval2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning BERT for German Named Entity Recognition

## Download and clean data for GermEval 2014 NER task

The following lines download the data set and convert it to a format compatible with CoNLL 2003.